# Pregunta 3: 
## Implementar un script Python para crear la estructura de la base de datos “Yapo-LosRios” (NB: pueden elegir el SGBD que desea)

In [67]:
import mysql.connector 

password = "password"
db_connection = mysql.connector.connect(user="root",host="localhost",password="password",  auth_plugin='mysql_native_password')
cursor = db_connection.cursor()
cursor.execute("CREATE DATABASE Yapo_LosRios_trabajoSebastianDuran;") # eliminar antes de entregar
cursor.execute("USE Yapo_LosRios_trabajoSebastianDuran")

In [66]:
cursor.execute("DROP DATABASE Yapo_LosRios_trabajoSebastianDuran;")

In [68]:
#tabla vendedor
cursor.execute("CREATE TABLE vendedor ("+
               "nombre VARCHAR(100) PRIMARY KEY, ciudad VARCHAR(30), codigo_region VARCHAR(3), "+
               "nombre_region VARCHAR(30), "+
               "fecha_inscripcion DATE)")

#tabla anuncio
cursor.execute("CREATE TABLE anuncio (url VARCHAR(300) PRIMARY KEY, "+
               "titulo VARCHAR(200), descripcion MEDIUMTEXT, precio VARCHAR(30), categoria VARCHAR(50), "
               +"nombre VARCHAR(100), FOREIGN KEY (nombre) REFERENCES vendedor(nombre),fecha_publicacion DATE)")

# Pregunta 4: 
## Implementar un script Python para llenar la base de datos “Yapo-LosRios” escrapeando algunos datos de Yapo.cl

In [69]:
from urllib.request import urlopen as uReq
import urllib.request

from bs4 import BeautifulSoup as soup
from selenium import webdriver
import time
from webdriver_manager.chrome import ChromeDriverManager

In [80]:
browser = webdriver.Chrome(ChromeDriverManager().install())
myUrl = 'https://www.yapo.cl/los_rios/todos_los_avisos?ca=11_s&l=0&w=1&cmn=243'
browser.get(myUrl)

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147


[WDM] - Driver [/home/sebastianduran2712/.wdm/drivers/chromedriver/linux64/84.0.4147.30/chromedriver] found in cache


In [81]:
pageSoup = soup(browser.page_source, 'html.parser')

pages = pageSoup.find('span',  {'class', 'nohistory FloatRight'}).a['href']

In [82]:
index = pages.rfind('=')
print(index)
lastPage = int(pages[index+1:])
print(lastPage)
root_pages = pages[:index+1]
print(root_pages)

46
621
https://www.yapo.cl/los_rios?ca=11_s&cmn=243&o=


In [83]:
import datetime

def convert_date(yapo_date):
    month=yapo_date.split(" ")[0]
    year=yapo_date.split(" ")[1]
    
    convert = {'Enero' : 1,
            'Febrero' : 2,
            'Marzo' : 3,
            'Abril' : 4,
            'Mayo' : 5,
            'Junio' : 6,
            'Julio' : 7,
            'Agosto' : 8,
            'Septiembre' : 9, 
            'Octubre' : 10,
            'Noviembre' : 11,
            'Diciembre' : 12
           }
    
    
    new_date=datetime.date(int(year), convert[month], 1) 

    return new_date


def convert_date_publication(yapo_date):
    day=yapo_date.split(" ")[0]
    month=yapo_date.split(" ")[1]
    hour=yapo_date.split(" ")[2]
    
    convert = {'Enero' : 1,
            'Febrero' : 2,
            'Marzo' : 3,
            'Abril' : 4,
            'Mayo' : 5,
            'Junio' : 6,
            'Julio' : 7,
            'Agosto' : 8,
            'Septiembre' : 9, 
            'Octubre' : 10,
            'Noviembre' : 11,
            'Diciembre' : 12
           }
    
    
    new_date=datetime.date(2020, convert[month], int(day) )
    return new_date
convert_date_publication("29 Agosto 17:08")

datetime.date(2020, 8, 29)

In [84]:

for i in range(20):
    
    #recuperarmos la URL de la pagina corriente
    url = root_pages + str(i+1)
    
    #con Selenium, vamos en navegar en esta página
    browser.get(url)
    
    #empezamos el scrapping de la pagina corriente
    pageSoup = soup(browser.page_source, 'html.parser')
    
    #recuperamos todos los tags HTML que corresponden a la lista de anuncios en esta pagina
    links = pageSoup.findAll('td', {'class' : 'thumbs_subject'})
    
    #empezamos a iterar sobre cada anuncio
    for link in links:
        
        #todos los datos que necesitamos encontrar
        url, titulo, descripcion, precio, categoria="","","","",""
        nombre, ciudad, codigo_region, nombre_region, fecha_inscripcion="","","","",""
        
        #Navegamos hacia la pagina del anuncio
        url=link.find('a',{'class':'title'})['href']
        print(url)
        browser.get(link.find('a',{'class':'title'})['href'])
        
        
        #RECUPERAMOS EL TITULO DEL ANUNCIO           
        pageSoup = soup(browser.page_source, 'html.parser')
        if(pageSoup.find('h1', {"id" : "da_subject"})):
            titulo = pageSoup.find('h1', {"id" : "da_subject"}).text.strip()
            print(titulo)
            
        #RECUPERAMOS LA DESCRIPCION DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "description"})):
            try:
                descripcion = pageSoup.find('div', {"class" : "description"}).text.split(' ', 1)[1].strip().replace(u'\n', u' ')
            except:
                continue
                
        #RECUPERAMOS EL PRECIO DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "price text-right"})):
            precio = pageSoup.find('div', {"class" : "price text-right"}).text.strip().replace(u'\n', u' ').replace(u'\t', u'')
            print("precio:"+precio)
            
        #RECUPERAMOS LA CATEGORIA DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "breadcrumbs"})):
            categoria = pageSoup.find('div', {"class" : "breadcrumbs"}).find('a', {"id" : "breadcrumb_category"}).find('strong').text.strip().replace(u'\n', u' ')
            print(categoria)
            
        #RECUPERAMOS LA FECHA DE PUBLICACIÓN DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "title"})):
            fecha_publicacion_raw = pageSoup.find('div', {"class" : "title"}).find('small', {"class" : "date"}).find('time').text.strip().replace(u'\n', u' ')
            fecha_publicacion = convert_date_publication(fecha_publicacion_raw)
            print(fecha_publicacion)
        
        #RECUPERAMOS EL NOMBRE DEL VENDEDOR, SU FECHA DE INSCRIPCION Y SU LOCALIDAD
        if(pageSoup.find('aside', {"class" : "sidebar-right"})):
            aside = pageSoup.find('aside', {"class" : "sidebar-right"})
    
            #print(aside.find('seller-info'))
            
            #NOMBRE
            nombre=aside.find('seller-info').attrs['username']
            
            #FECHA DE INSCRIPCION
            fecha_inscripcion_raw=aside.find('seller-info').attrs['seniority']
            try:
                fecha_inscripcion_raw=fecha_inscripcion_raw[len("En Yapo desde "):]
                fecha_inscripcion=convert_date(fecha_inscripcion_raw)
                print(fecha_inscripcion)
            except:
                fecha_inscripcion=None
                continue
                
            #LOCALIDAD
            localidad_raw=aside.find('seller-info').attrs['region']
            
            #print(localidad_raw)
            
            #region_raw=localidad.split(",")[0]
            #ciudad_raw=localidad.split(",")[1]
            
            region_raw=localidad_raw.split(",")[0]
            ciudad_raw=localidad_raw.split(",")[1]
            
            codigo_region=region_raw.split(" ")[0]
            nombre_region=region_raw[len(codigo_region)+1:]
            ciudad=ciudad_raw[1:]
            
            print(nombre)
            print(codigo_region)
            print(nombre_region)
            print(ciudad)
         
        
        # LLENAMOS LA BASE DE DATOS: TABLA VENDEDOR
        
        try:
            sql = "INSERT INTO vendedor (nombre, ciudad, codigo_region, nombre_region, fecha_inscripcion) VALUES (%s, %s, %s,%s,%s)"
            val = (nombre, ciudad, codigo_region, nombre_region, fecha_inscripcion)
            cursor.execute(sql, val)
        except:
            continue
        
        # LLENAMOS LA BASE DE DATOS: TABLA ANUNCIO
        try:
            sql = "INSERT INTO anuncio (url, titulo, descripcion, precio, categoria,nombre,fecha_publicacion) VALUES (%s, %s, %s,%s,%s, %s, %s)"
            val = (url, titulo, descripcion, precio, categoria,nombre,fecha_publicacion)
            cursor.execute(sql, val)
        except:
            print("no ingresado")
            continue
        
        cursor.execute("COMMIT") 

https://www.yapo.cl/los_rios/comprar/casa_en_villa_el_estanque_74023169.htm?ca=11_s&first=1&oa=74023169&xsp=0
casa en Villa el Estanque
precio:UF 4.182,00
Vendo - Casa
2020-08-28
2020-08-01
Yennifer
XIV
Los Ríos
Valdivia
https://www.yapo.cl/los_rios/calzado/zapatos_num_25_74023063.htm?ca=11_s&oa=74023063&xsp=1
Zapatos núm 25
precio:$ 5.000
Calzado
2020-08-28
2015-12-01
Marisol Parada Caro
XIV
Los Ríos
Valdivia
https://www.yapo.cl/los_rios/calzado/zapatillas__74023052.htm?ca=11_s&oa=74023052&xsp=2
Zapatillas
precio:$ 35.000
Calzado
2020-08-28
2018-04-01
Geraldine Fuentealba 
XIV
Los Ríos
Valdivia
https://www.yapo.cl/los_rios/vestuario-futura-mama-ninos/camisa_nino_talla_6_74023046.htm?ca=11_s&oa=74023046&xsp=3
Camisa niño talla 6
precio:$ 3.000
Vestuario futura mamá y niños
2020-08-28
2020-08-01
ventas.detodounpoco.sanjose
XIV
Los Ríos
Valdivia
https://www.yapo.cl/los_rios/ofertas_de_empleo/tens_o_auxiliar_de_enfermeria_74023044.htm?ca=11_s&oa=74023044&xsp=4
Tens o auxiliar de enfermerí

Tortas o algo dulce para la once
precio:$ 5.000
Servicios
2020-08-28
2016-10-01
Carolina
XIV
Los Ríos
Valdivia
https://www.yapo.cl/los_rios/celulares/iphone_6s__74021787.htm?ca=11_s&oa=74021787&xsp=37
IPhone 6s
Celulares, teléfonos y accesorios
2020-08-28
https://www.yapo.cl/los_rios/coches-articulos-infantiles/shampo_y_acondicionador_natura_74021754.htm?ca=11_s&oa=74021754&xsp=38
Shampo y acondicionador natura
precio:$ 4.000
Coches y artículos infantiles
2020-08-28
2018-07-01
Antonia
XIV
Los Ríos
Valdivia
https://www.yapo.cl/los_rios/negocios_maquinaria_construccion/caldera_a_lena_74021753.htm?ca=11_s&oa=74021753&xsp=39
Caldera a leña
precio:$ 300.000
Negocios, maquinaria y construcción
2020-08-28
2014-05-01
veronica
XIV
Los Ríos
Valdivia
https://www.yapo.cl/los_rios/comprar/parcela_linguento_valdivia_70422671.htm?ca=11_s&oa=70422671&xsp=40
PARCELA LINGUENTO Valdivia
precio:UF 1.100,00
Vendo - Terreno
2020-08-28
2020-08-01
Berrios Zegers Propiedades
XIV
Los Ríos
Valdivia
https://www.y

Fletes y mudanzas Valdivia
Servicios
2020-08-28
2017-04-01
Jaime Roa
XIV
Los Ríos
Valdivia
https://www.yapo.cl/los_rios/vestuario-futura-mama-ninos/ropa_nina_talla_4_74019968.htm?ca=11_s&oa=74019968&xsp=25
Ropa niña talla 4
precio:$ 25.000
Vestuario futura mamá y niños
2020-08-28
2015-04-01
Paula
XIV
Los Ríos
Valdivia
https://www.yapo.cl/los_rios/busco_empleo/empleada_de_aseo_74019962.htm?ca=11_s&oa=74019962&xsp=26
empleada de aseo
precio:$ 250.000
Busco empleo
2020-08-28
2020-08-01
Natali magdalena 
XIV
Los Ríos
Valdivia
https://www.yapo.cl/los_rios/computadores/disco_duro_hdd_samsung_750gb_sata_3_5400rpm_74019886.htm?ca=11_s&oa=74019886&xsp=27
Disco Duro HDD Samsung 750GB SATA 3 5400rpm
precio:$ 20.000
Computadores y accesorios
2020-08-28
2013-12-01
Hugo Orellana
XIV
Los Ríos
Valdivia
https://www.yapo.cl/los_rios/ofertas_de_empleo/persona_con_experiencia_en_ruta_74019846.htm?ca=11_s&oa=74019846&xsp=28
Persona con experiencia en ruta
Ofertas de empleo
2020-08-28
2015-09-01
Juan pablo


KeyboardInterrupt: 

# Pregunta 5: 
## 5.1- ¿Cuál es el precio promedio de los anuncios por categoría?


In [85]:
# buscar todos las categorias sin repetición
cursor.execute("SELECT DISTINCT categoria FROM anuncio;")
x = cursor.fetchall()

In [87]:
for i in x:
    y = "SELECT precio FROM anuncio WHERE categoria = '"+i[0]+"';"
    print(i[0])
    cursor.execute(y)
    precios = cursor.fetchall()
    precio = 0; cantidad = 0
    for j in precios:
        z = j[0][2::].replace('.','')
        z=z.split(",")[0]
        if z != '':
            precio += int(z)
            cantidad += 1
    if cantidad != 0:
        print(precio/cantidad)
    else:
        print("****  Sin información de presios    ****")

Accesorios y piezas para vehículos
140000.0
Calzado
35000.0
Vendo - Casa
4182.0
Ofertas de empleo
400000.0


## 5.2- ¿Cuál es el número de anuncios por vendedor?

In [88]:
cursor.execute("SELECT  nombre, count(anuncio.url)"
               +"FROM anuncio GROUP BY nombre;")
x = cursor.fetchall()
for i in x:
    print(i)

('Alejandra Ernest', 1)
('Geraldine Fuentealba ', 1)
('sandra', 1)
('Yennifer', 1)


## 5.3- Insertar una columna “Perfil” en la tabla Vendedor. Los Vendedores que publicaron más de 5 anuncios en el último día, tienen un perfil “Pro”, los otros se consideran “Personal”.

In [101]:
try:
    cursor.execute(" ALTER TABLE vendedor ADD perfil char(8)")
excep
    print("listo")

listo


In [90]:
for i in x :
    if i[1]<5:
        cursor.execute("UPDATE vendedor SET perfil = 'personal' WHERE nombre='" + i[0]+"';")
    else:
        cursor.execute("UPDATE vendedor SET perfil = 'pro' WHERE nombre='" + i[0]+"';")
cursor.execute("COMMIT")

## 5.4-¿Cuál es el precio promedio de los anuncios según el día de la semana (lunes, martes, miercoles, etc.)?

In [112]:
cursor.execute("select DISTINCT DAYNAME(fecha_publicacion) from anuncio;")
nombresDias = cursor.fetchall()
cursor.execute("select DAYNAME(fecha_publicacion), precio from anuncio;")
p = cursor.fetchall()
for i in nombresDias:
    suma = 0
    count = 0
    for j in p:
        if j[0]==i[0]:
            z = j[1][2::].replace('.','')
            z=z.split(",")[0]
            suma+= int(z)
            count +=1
    suma/=count
    print(i[0] + " ",suma)

Friday  144795.5


# Pregunta 6: 
## Implementar un script Python para visualizar la distribución de los precios en la categoría “Arriendo - Departamento”.

In [114]:
cursor.execute("SELECT precio from anuncio WHERE categoria='Arriendo - Departamento';")
y = cursor.fetchall()
print(y)

[]
